In [1]:
import pandas as pd
from pandas_profiling import ProfileReport as pr
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
import requests
import json
from sklearn.linear_model import LinearRegression

In [2]:
train_data = pd.read_csv("Challenge1_train_data.csv")

In [3]:
train_data.head()

,date,dc_name,size_code,retail_price,total_tires,zip_code
0,2022-02-05,OAKLAND,1856015,70.090,1,94604
1,2021-03-02,OAKLAND,1856015,59.090,1,94604
2,2020-12-08,OAKLAND,1856015,53.545,1,94604
3,2022-01-17,OAKLAND,1856015,58.410,1,94604
4,2020-11-13,OAKLAND,1856015,43.085,1,94604


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284700 entries, 0 to 284699
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   date          284700 non-null  object 
 1   dc_name       284700 non-null  object 
 2   size_code     284700 non-null  int64  
 3   retail_price  284666 non-null  float64
 4   total_tires   284700 non-null  int64  
 5   zip_code      284700 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 13.0+ MB


In [5]:
train_data.date.min(),train_data.date.max()

('2020-09-20', '2022-09-19')

In [6]:
test_date = parse(train_data.date.max()) + relativedelta(days=-14)
str(test_date)

'2022-09-05 00:00:00'

In [7]:
profile = pr(train_data, title="ATD Data Analysis", explorative=True)
profile.to_file("atd_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/akhileshpothuri/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:4812: RuntimeWarning: overflow encountered in long_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
train_data[train_data['retail_price']==0]

,date,dc_name,size_code,retail_price,total_tires,zip_code
236727,2021-01-17,OAKLAND,1856015,0.0,0,94604
236728,2022-03-20,OAKLAND,1856015,0.0,0,94604
236729,2020-09-29,OAKLAND,1856015,0.0,0,94604
236730,2021-07-13,OAKLAND,1856015,0.0,0,94604
236731,2022-04-06,OAKLAND,1856015,0.0,0,94604
...,...,...,...,...,...,...
284695,2022-07-24,BAKERSFIELD,3512520,0.0,0,93308
284696,2020-11-22,BAKERSFIELD,3512520,0.0,0,93308
284697,2021-08-08,BAKERSFIELD,3512520,0.0,0,93308
284698,2022-02-14,BAKERSFIELD,3512520,0.0,0,93308


In [9]:
len(train_data)

284700

In [10]:
###here we are getting the weather data from the api into a dataframe for all the cities


weather_sacramento_url = f'https://archive-api.open-meteo.com/v1/era5?latitude=38.5816&longitude=-121.4944&start_date=2020-09-20&end_date=2022-09-19&timezone=GMT&daily=temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max' 
weather_bakersfield_url= f'https://archive-api.open-meteo.com/v1/era5?latitude=35.3733&longitude=-119.0187&start_date=2020-09-20&end_date=2022-09-19&timezone=GMT&daily=temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max'
weather_sanjose_url = f'https://archive-api.open-meteo.com/v1/era5?latitude=37.3387&longitude=-121.8853&start_date=2020-09-20&end_date=2022-09-19&timezone=GMT&daily=temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max'
weather_oakland_url = f'https://archive-api.open-meteo.com/v1/era5?latitude=37.8044&longitude=-122.2712&start_date=2020-09-20&end_date=2022-09-19&timezone=GMT&daily=temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max'


def get_weather_data(url,city_name):
    weather_resp = requests.get(url)
    a=weather_resp.json()
    weather=pd.DataFrame.from_dict(a.get('daily'))
    weather['city_name']=city_name
    return weather

weather_sacramento=get_weather_data(weather_sacramento_url,'SACRAMENTO')
print(weather_sacramento.info())

weather_bakersfield=get_weather_data(weather_bakersfield_url,'BAKERSFIELD')
print(weather_bakersfield.info())
weather_sanjose=get_weather_data(weather_sanjose_url,'SAN JOSE')
print(weather_sanjose.info())
weather_oakland= get_weather_data(weather_oakland_url,'OAKLAND')
print(weather_oakland.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   time                730 non-null    object 
 1   temperature_2m_max  730 non-null    float64
 2   temperature_2m_min  730 non-null    float64
 3   precipitation_sum   730 non-null    float64
 4   windspeed_10m_max   730 non-null    float64
 5   city_name           730 non-null    object 
dtypes: float64(4), object(2)
memory usage: 34.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   time                730 non-null    object 
 1   temperature_2m_max  730 non-null    float64
 2   temperature_2m_min  730 non-null    float64
 3   precipitation_sum   730 non-null    float64
 4   windspeed_10m_max   730 non-null    float64

In [11]:
weather_data = pd.concat([weather_sacramento, weather_bakersfield,weather_sanjose,weather_oakland],ignore_index=True)
weather_data.head()

,time,temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max,city_name
0,2020-09-20,33.3,15.3,0.0,8.5,SACRAMENTO
1,2020-09-21,33.8,16.4,0.0,10.7,SACRAMENTO
2,2020-09-22,32.3,17.0,0.0,12.3,SACRAMENTO
3,2020-09-23,32.7,15.5,0.0,11.8,SACRAMENTO
4,2020-09-24,33.1,17.3,0.0,12.5,SACRAMENTO


In [12]:
train_data = train_data.merge(weather_data, how='left', left_on=['date', 'dc_name'], right_on=['time', 'city_name']).drop(columns=['city_name','time'])
train_data.head()

,date,dc_name,size_code,retail_price,total_tires,zip_code,temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max
0,2022-02-05,OAKLAND,1856015,70.090,1,94604,17.5,4.8,0.0,7.4
1,2021-03-02,OAKLAND,1856015,59.090,1,94604,20.3,6.3,0.0,12.9
2,2020-12-08,OAKLAND,1856015,53.545,1,94604,23.6,5.6,0.0,10.0
3,2022-01-17,OAKLAND,1856015,58.410,1,94604,15.8,6.8,0.0,7.4
4,2020-11-13,OAKLAND,1856015,43.085,1,94604,16.9,7.5,0.0,21.5


## Exploratory Data Anlaysis

In [13]:
zero_data = train_data[train_data['retail_price']==0]
grouped_size_data = zero_data.groupby(['dc_name','size_code'])['date'].count()
grouped_size_data

dc_name      size_code
BAKERSFIELD  1856015      128
             1856514      181
             1856515      118
             1956015      115
             1956515      115
                         ... 
SAN JOSE     2757018      159
             2854522      127
             2857017      207
             3512520       97
             22570195     182
Name: date, Length: 390, dtype: int64

In [14]:
grouped_dc_data = zero_data.groupby(['dc_name'])['date'].count()
grouped_dc_data

dc_name
BAKERSFIELD    13663
OAKLAND        11282
SACRAMENTO     15428
SAN JOSE        7600
Name: date, dtype: int64

## Data Classification

In [15]:
train_data['Key'] = train_data['dc_name'] +'|' +train_data['size_code'].astype(str)

In [16]:
train_data['date'] = pd.to_datetime(train_data['date'])
train_data.head()

,date,dc_name,size_code,retail_price,total_tires,zip_code,temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max,Key
0,2022-02-05,OAKLAND,1856015,70.090,1,94604,17.5,4.8,0.0,7.4,OAKLAND|1856015
1,2021-03-02,OAKLAND,1856015,59.090,1,94604,20.3,6.3,0.0,12.9,OAKLAND|1856015
2,2020-12-08,OAKLAND,1856015,53.545,1,94604,23.6,5.6,0.0,10.0,OAKLAND|1856015
3,2022-01-17,OAKLAND,1856015,58.410,1,94604,15.8,6.8,0.0,7.4,OAKLAND|1856015
4,2020-11-13,OAKLAND,1856015,43.085,1,94604,16.9,7.5,0.0,21.5,OAKLAND|1856015


In [17]:
train_df = train_data[train_data['date']<str(test_date)]

In [18]:
cv_data = train_df.groupby(['Key']).agg(average=('total_tires','mean'),
                                                    sd=('total_tires','std')).reset_index()
#cv_data['key_adi']= cv_data['dc_name']+'|'+cv_data['size_code'].astype(str)

In [19]:
## Calculating CV_squared

cv_data['cv_sqr'] = (cv_data['sd']/cv_data['average'])**2
cv_data

,Key,average,sd,cv_sqr
0,BAKERSFIELD|1856015,9.195804,7.457122,0.657602
1,BAKERSFIELD|1856514,5.411189,5.668122,1.097218
2,BAKERSFIELD|1856515,12.304895,10.371903,0.710495
3,BAKERSFIELD|1956015,28.994406,20.119600,0.481515
4,BAKERSFIELD|1956515,34.744056,23.725029,0.466286
...,...,...,...,...
385,SAN JOSE|2756520,5.565035,5.771771,1.075678
386,SAN JOSE|2757018,7.676923,6.810600,0.787039
387,SAN JOSE|2854522,7.345455,6.092618,0.687972
388,SAN JOSE|2857017,6.467133,6.428880,0.988205


In [20]:
#train_df['key_adi'] = train_df['dc_name']+'|'+train_df['size_code'].astype(str)

non_zero_df = train_df[train_df['total_tires']>0]
non_zero_df = non_zero_df.dropna(subset=('total_tires'))
tire_by_date= non_zero_df.groupby(['Key','date']).agg(count=('Key','count')).reset_index()

In [21]:
skus=tire_by_date.Key.value_counts()
skus

SACRAMENTO|1956515    702
SACRAMENTO|2155517    701
SACRAMENTO|2055516    696
SACRAMENTO|2156016    695
SACRAMENTO|2256517    693
                     ... 
SAN JOSE|2056016      491
SACRAMENTO|2356016    489
SACRAMENTO|2154018    489
SAN JOSE|2555020      487
SACRAMENTO|11225      483
Name: Key, Length: 390, dtype: int64

In [22]:
new_df= pd.DataFrame()
for i in range(len(skus.index)):
    a= tire_by_date[tire_by_date['Key']==skus.index[i]]
    a['previous_date']=a['date'].shift(1)
    new_df=pd.concat([new_df,a],axis=0)

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

...

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['previous_date']=a['date'].shift(1)
/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_36020/3957170539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [23]:
new_df['duration']=new_df['date']- new_df['previous_date']

In [24]:
new_df.head()

,Key,date,count,previous_date,duration
117189,SACRAMENTO|1956515,2020-09-20,1,NaT,NaT
117190,SACRAMENTO|1956515,2020-09-21,1,2020-09-20,1 days
117191,SACRAMENTO|1956515,2020-09-22,1,2020-09-21,1 days
117192,SACRAMENTO|1956515,2020-09-23,1,2020-09-22,1 days
117193,SACRAMENTO|1956515,2020-09-24,1,2020-09-23,1 days


In [25]:
new_df['Duration']=new_df['duration'].astype(str).str.replace('days','')
new_df['Duration']=pd.to_numeric(new_df['Duration'],errors='coerce')
ADI = new_df.groupby('Key').agg(ADI = ('Duration','mean')).reset_index()
ADI

,Key,ADI
0,BAKERSFIELD|1856015,1.206780
1,BAKERSFIELD|1856514,1.328358
2,BAKERSFIELD|1856515,1.186667
3,BAKERSFIELD|1956015,1.182724
4,BAKERSFIELD|1956515,1.182724
...,...,...
385,SAN JOSE|2756520,1.390625
386,SAN JOSE|2757018,1.275986
387,SAN JOSE|2854522,1.206430
388,SAN JOSE|2857017,1.385214


In [26]:
adi_cv=pd.merge(ADI,cv_data)
adi_cv

,Key,ADI,average,sd,cv_sqr
0,BAKERSFIELD|1856015,1.206780,9.195804,7.457122,0.657602
1,BAKERSFIELD|1856514,1.328358,5.411189,5.668122,1.097218
2,BAKERSFIELD|1856515,1.186667,12.304895,10.371903,0.710495
3,BAKERSFIELD|1956015,1.182724,28.994406,20.119600,0.481515
4,BAKERSFIELD|1956515,1.182724,34.744056,23.725029,0.466286
...,...,...,...,...,...
385,SAN JOSE|2756520,1.390625,5.565035,5.771771,1.075678
386,SAN JOSE|2757018,1.275986,7.676923,6.810600,0.787039
387,SAN JOSE|2854522,1.206430,7.345455,6.092618,0.687972
388,SAN JOSE|2857017,1.385214,6.467133,6.428880,0.988205


In [27]:
def category(df):
    a=0
    
    if((df['ADI']<=1.34) & (df['cv_sqr']<=0.49)):
        a='Smooth'
    if((df['ADI']>=1.34) & (df['cv_sqr']>=0.49)):  
        a='Lumpy'
    if((df['ADI']<1.34) & (df['cv_sqr']>0.49)):
        a='Erratic'
    if((df['ADI']>1.34) & (df['cv_sqr']<0.49)):
        a='Intermittent'
    return a

In [28]:
adi_cv['category']=adi_cv.apply(category,axis=1)
adi_cv[['Key','category']]

,Key,category
0,BAKERSFIELD|1856015,Erratic
1,BAKERSFIELD|1856514,Erratic
2,BAKERSFIELD|1856515,Erratic
3,BAKERSFIELD|1956015,Smooth
4,BAKERSFIELD|1956515,Smooth
...,...,...
385,SAN JOSE|2756520,Lumpy
386,SAN JOSE|2757018,Erratic
387,SAN JOSE|2854522,Erratic
388,SAN JOSE|2857017,Lumpy


In [29]:
adi_cv.category.value_counts()


Erratic    246
Smooth      98
Lumpy       46
Name: category, dtype: int64

In [30]:
train_data = train_data.merge(adi_cv[['Key','category']],on=['Key'],how='left')

In [31]:
train_data

,date,dc_name,size_code,retail_price,total_tires,zip_code,temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max,Key,category
0,2022-02-05,OAKLAND,1856015,70.090,1,94604,17.5,4.8,0.0,7.4,OAKLAND|1856015,Lumpy
1,2021-03-02,OAKLAND,1856015,59.090,1,94604,20.3,6.3,0.0,12.9,OAKLAND|1856015,Lumpy
2,2020-12-08,OAKLAND,1856015,53.545,1,94604,23.6,5.6,0.0,10.0,OAKLAND|1856015,Lumpy
3,2022-01-17,OAKLAND,1856015,58.410,1,94604,15.8,6.8,0.0,7.4,OAKLAND|1856015,Lumpy
4,2020-11-13,OAKLAND,1856015,43.085,1,94604,16.9,7.5,0.0,21.5,OAKLAND|1856015,Lumpy
...,...,...,...,...,...,...,...,...,...,...,...,...
284695,2022-07-24,BAKERSFIELD,3512520,0.000,0,93308,36.0,21.1,0.0,8.0,BAKERSFIELD|3512520,Erratic
284696,2020-11-22,BAKERSFIELD,3512520,0.000,0,93308,17.7,4.6,0.0,9.4,BAKERSFIELD|3512520,Erratic
284697,2021-08-08,BAKERSFIELD,3512520,0.000,0,93308,35.9,21.4,0.0,8.7,BAKERSFIELD|3512520,Erratic
284698,2022-02-14,BAKERSFIELD,3512520,0.000,0,93308,23.9,6.8,0.0,9.7,BAKERSFIELD|3512520,Erratic


### Handling Missing Values

In [32]:
train_data[train_data['retail_price'].isna()]

,date,dc_name,size_code,retail_price,total_tires,zip_code,temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max,Key,category
77863,2020-12-03,SACRAMENTO,1757014,NaN,4,95838,18.4,1.8,0.0,8.1,SACRAMENTO|1757014,Erratic
81565,2022-04-04,SACRAMENTO,1857014,NaN,2,95838,28.6,11.6,0.0,18.2,SACRAMENTO|1857014,Erratic
82169,2021-09-17,SACRAMENTO,1955515,NaN,1,95838,29.2,12.3,0.0,13.6,SACRAMENTO|1955515,Lumpy
84743,2020-11-12,SACRAMENTO,1957014,NaN,2,95838,17.3,2.3,0.0,5.4,SACRAMENTO|1957014,Erratic
93686,2022-04-04,SACRAMENTO,2154018,NaN,4,95838,28.6,11.6,0.0,18.2,SACRAMENTO|2154018,Lumpy
97510,2020-10-23,SACRAMENTO,2155518,NaN,3,95838,26.3,12.2,0.0,12.3,SACRAMENTO|2155518,Erratic
97587,2021-11-01,SACRAMENTO,2155518,NaN,4,95838,19.0,13.7,3.8,10.0,SACRAMENTO|2155518,Erratic
98654,2020-11-16,SACRAMENTO,2156017,NaN,2,95838,21.0,6.9,0.0,6.5,SACRAMENTO|2156017,Erratic
100598,2022-05-23,SACRAMENTO,2157015,NaN,2,95838,36.5,17.1,0.0,14.0,SACRAMENTO|2157015,Erratic
102646,2020-12-11,SACRAMENTO,2158516,NaN,4,95838,18.8,4.4,0.0,10.9,SACRAMENTO|2158516,Erratic


In [33]:
# Impute the missing values for the retail_price
df_sacramento = train_data[train_data['dc_name']=='SACRAMENTO']

sac_sizecode = df_sacramento[df_sacramento['retail_price'].isna()]['size_code'].tolist()

for code in sac_sizecode : 
    # Get the mean value
    mean_value = df_sacramento[(df_sacramento['size_code'] == code) & ((~df_sacramento['retail_price'].isna()) & (df_sacramento['retail_price'] != 0))]['retail_price'].mean()
    df_sacramento.loc[(df_sacramento['size_code'] == code) & (df_sacramento['retail_price'].isna()), 'retail_price'] = mean_value

In [34]:
# Putting the sacramento data into the main dataframe after imputing
indexes_to_drop = train_data[train_data['dc_name']=='SACRAMENTO'].index
train_data.drop(index= indexes_to_drop, inplace = True)
train_data = pd.concat([
    train_data,df_sacramento
], ignore_index= True)

In [35]:
train_data['day_name'] = train_data['date'].dt.day_name()

In [36]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284700 entries, 0 to 284699
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   date                284700 non-null  datetime64[ns]
 1   dc_name             284700 non-null  object        
 2   size_code           284700 non-null  int64         
 3   retail_price        284700 non-null  float64       
 4   total_tires         284700 non-null  int64         
 5   zip_code            284700 non-null  int64         
 6   temperature_2m_max  284700 non-null  float64       
 7   temperature_2m_min  284700 non-null  float64       
 8   precipitation_sum   284700 non-null  float64       
 9   windspeed_10m_max   284700 non-null  float64       
 10  Key                 284700 non-null  object        
 11  category            284700 non-null  object        
 12  day_name            284700 non-null  object        
dtypes: datetime64[ns](1), float64

## Univariate Models

In [37]:
uni_train_data = train_data[['Key','dc_name','size_code','date','total_tires']].sort_values(['Key','dc_name','size_code','date'])
uni_train_data

,Key,dc_name,size_code,date,total_tires
157968,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-20,0
76544,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-21,18
76183,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-22,7
76064,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-23,3
76485,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-24,15
...,...,...,...,...,...
75002,SAN JOSE|3512520,SAN JOSE,3512520,2022-09-15,8
75018,SAN JOSE|3512520,SAN JOSE,3512520,2022-09-16,10
157638,SAN JOSE|3512520,SAN JOSE,3512520,2022-09-17,0
157665,SAN JOSE|3512520,SAN JOSE,3512520,2022-09-18,0


In [38]:
uni_train_data['date'] = pd.to_datetime(uni_train_data['date'])
uni_train_data.head()

,Key,dc_name,size_code,date,total_tires
157968,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-20,0
76544,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-21,18
76183,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-22,7
76064,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-23,3
76485,BAKERSFIELD|1856015,BAKERSFIELD,1856015,2020-09-24,15


In [42]:
def model_training(sliced_data):
    train_df = uni_train_data[uni_train_data['date']<str(test_date)][['date','total_tires']]
    test_df = uni_train_data[uni_train_data['date']>=str(test_date)][['date','total_tires']]
    
    train_df=train_df.reset_index()
    test_df=test_df.reset_index()
    x_train = train_df['date'].values.reshape(-1, 1)
    x_train = x_train.astype(float)
    y_train = train_df['total_tires'].values
    
    x_test = test_df['date'].values.reshape(-1, 1)
    x_test = x_test.astype(float)
    t_test = test_df['total_tires'].values
    
    
    
    regressor = LinearRegression() 
        
    regressor.fit(x_train,y_train)
        

    y_test = regressor.predict(x_test)
    test_df['Forecast'] = y_test.round()
    test_df['Model'] = 'Linear_Regression'
    return test_df
    

In [ ]:
result_df = uni_train_data.groupby('Key').apply(model_training).reset_index()
result_df = uni_train_data.groupby('Key').apply(model_training).reset_index()
result_df = result_df[['Key','date','total_tires','Forecast','Model']]

result_df.reset_index()

In [ ]:
result_df.date.unique()